In [ ]:
# 运行在colab上需要使用，在其他环境请删除
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

%cd drive/My\ Drive/AI/Lab5

Mounted at /content/drive
/content/drive/My Drive/AI/Lab5


In [ ]:
%ls Lab5_Data

data/  test_without_label.txt  train.txt


In [ ]:
# colab上需要使用pip安装其他版本的torchtext，如果使用requirements.txt，请去除
!pip uninstall torchtext -y
!pip install torchtext==0.6.0
import torch
from torchtext.data import Field, LabelField
from torchtext.data import TabularDataset

Found existing installation: torchtext 0.6.0
Uninstalling torchtext-0.6.0:
  Successfully uninstalled torchtext-0.6.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached torchtext-0.6.0-py3-none-any.whl (64 kB)


In [ ]:
# 标签转为数字
label_to_index = {'negative':0, 'neutral':1, 'positive':2}
index_to_label = {0:'negative', 1:'neutral', 2:'positive'}

In [ ]:
# 使用torchtext读取文本
TEXT = Field(sequential=True, lower=True, fix_length=30, use_vocab=True)
LABEL = LabelField(sequential=False, use_vocab=False, dtype = torch.long)
ID = Field(sequential=False, lower=False, fix_length=5, use_vocab=False, dtype = torch.float)

fields = {'id': ('id',ID), 'text': ('text',TEXT),'label': ('label',LABEL)}
train_data, valid_data = TabularDataset.splits(
    path='jsonl',
    train='train.jsonl',
    validation='valid.jsonl',
    format='json',
    fields=fields)

test_fields = {'id': ('id',ID), 'text': ('text',TEXT)}
test_data = TabularDataset(
    path='jsonl/test.jsonl',
    format='json',
    fields=test_fields
)


In [ ]:
print(train_data.fields)
print(valid_data.fields)
print(test_data.fields)

{'id': <torchtext.data.field.Field object at 0x7fafd6be81d0>, 'text': <torchtext.data.field.Field object at 0x7fafd6be80d0>, 'label': <torchtext.data.field.LabelField object at 0x7fafd6be8190>}
{'id': <torchtext.data.field.Field object at 0x7fafd6be81d0>, 'text': <torchtext.data.field.Field object at 0x7fafd6be80d0>, 'label': <torchtext.data.field.LabelField object at 0x7fafd6be8190>}
{'id': <torchtext.data.field.Field object at 0x7fafd6be81d0>, 'text': <torchtext.data.field.Field object at 0x7fafd6be80d0>}


In [ ]:
print(len(train_data.examples))
print(len(valid_data.examples))
print(len(test_data.examples))
print(train_data.examples[0].text)
print(valid_data.examples[0].label)
print(test_data.examples[0].id)
print(type(test_data.examples[0].id))

3202
794
511
['rt', 'amitswami77', 'the', 'conspirators', 'evil', 'eye', 'set', 'physically', 'attack', 'asaram', 'bapu', 'ji', 'wedemandsafety4bapuji', 'http', '//t.co/n8']
2
8
<class 'str'>


In [ ]:
# 用训练数据构建词典
from torchtext.vocab import Vectors
from torch.utils.data import DataLoader
TEXT.build_vocab(train_data, 
                  max_size=50000, 
                  vectors='glove.6B.300d', 
                  unk_init=torch.Tensor.normal_ 
)

print(TEXT.vocab.freqs.most_common(20))
print(TEXT.vocab.itos[:10])
print(TEXT.vocab.stoi)

[('rt', 1373), ('http', 1186), ('i', 320), ("'s", 282), ('...', 168), ('the', 151), ('``', 149), ("''", 146), ('love', 128), ("n't", 113), ('new', 110), ('happy', 100), ('day', 90), ('black', 85), ('today', 81), ('like', 81), ('this', 79), ('get', 74), ('take', 69), ("'m", 68)]
['<unk>', '<pad>', 'rt', 'http', 'i', "'s", '...', 'the', '``', "''"]
defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7fafd66d8fd0>>, {'<unk>': 0, '<pad>': 1, 'rt': 2, 'http': 3, 'i': 4, "'s": 5, '...': 6, 'the': 7, '``': 8, "''": 9, 'love': 10, "n't": 11, 'new': 12, 'happy': 13, 'day': 14, 'black': 15, 'like': 16, 'today': 17, 'this': 18, 'get': 19, 'take': 20, "'m": 21, 'one': 22, 'a': 23, 'my': 24, 'https': 25, 'why': 26, 'so': 27, 'got': 28, 'passionate': 29, 'quiz': 30, '2': 31, 'night': 32, 'time': 33, 'finished': 34, 'it': 35, '..': 36, 'see': 37, 'you': 38, 'de': 39, 'when': 40, 'we': 41, 'much': 42, 'still': 43, "'re": 44, 'abandoned': 45, 'excited': 46, 'life': 

In [ ]:
print(TEXT.vocab.vectors.size())

torch.Size([13624, 300])


In [ ]:
# 将文本处理后，读取到迭代器，用于训练输入
from torchtext.data import Iterator
device = 'cuda'
train_iterator = Iterator(
                  train_data,
                  batch_size=16,
                  train=True,
                  device=device)
valid_iterator = Iterator(
                  valid_data,
                  batch_size=16,
                  train=False,
                  sort=False,
                  device=device)
test_iterator = Iterator(
                  test_data,
                  batch_size=16,
                  train=False,
                  sort=False,
                  device=device)
for batch in train_iterator:
    print(batch.label)
#for batch in test_iterator:
#    print(batch.text)

tensor([2, 2, 0, 2, 1, 0, 2, 2, 2, 0, 2, 2, 1, 0, 1, 2], device='cuda:0')
tensor([2, 2, 0, 0, 0, 2, 1, 0, 2, 2, 2, 2, 1, 2, 2, 2], device='cuda:0')
tensor([1, 2, 0, 2, 2, 0, 0, 0, 2, 2, 2, 2, 2, 2, 1, 2], device='cuda:0')
tensor([1, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2], device='cuda:0')
tensor([2, 2, 2, 0, 2, 0, 0, 2, 0, 0, 2, 2, 0, 2, 0, 2], device='cuda:0')
tensor([2, 0, 0, 2, 2, 0, 2, 2, 1, 2, 2, 0, 2, 2, 2, 2], device='cuda:0')
tensor([2, 2, 0, 2, 1, 2, 0, 0, 2, 2, 1, 2, 1, 2, 2, 2], device='cuda:0')
tensor([0, 2, 2, 2, 2, 2, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2], device='cuda:0')
tensor([0, 0, 0, 1, 0, 2, 0, 2, 0, 2, 2, 2, 0, 1, 2, 0], device='cuda:0')
tensor([2, 2, 2, 0, 0, 1, 2, 0, 2, 1, 0, 2, 2, 1, 2, 2], device='cuda:0')
tensor([2, 2, 2, 0, 0, 2, 1, 2, 2, 2, 2, 2, 0, 2, 0, 0], device='cuda:0')
tensor([1, 2, 2, 0, 0, 2, 1, 2, 2, 2, 0, 2, 2, 2, 2, 1], device='cuda:0')
tensor([2, 0, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2], device='cuda:0')
tensor([2, 2, 2, 0, 1, 2, 1, 2, 0, 2, 

In [ ]:
%ls
%ls Lab5_Data
%ls Lab5_Data/data

data_mining.ipynb  jsonl/  Lab5_Data/  main.ipynb  model/
data/  test_without_label.txt  train.txt
1000.jpg  154.jpg   2098.jpg  2849.jpg  340.jpg   3977.jpg  4524.jpg  5072.jpg
1000.txt  154.txt   2098.txt  2849.txt  340.txt   3977.txt  4524.txt  5072.txt
1001.jpg  1550.jpg  2099.jpg  284.jpg   3410.jpg  3978.jpg  4525.jpg  5073.jpg
1001.txt  1550.txt  2099.txt  284.txt   3410.txt  3978.txt  4525.txt  5073.txt
1002.jpg  1551.jpg  209.jpg   2850.jpg  3411.jpg  3979.jpg  4526.jpg  5074.jpg
1002.txt  1551.txt  209.txt   2850.txt  3411.txt  3979.txt  4526.txt  5074.txt
1003.jpg  1552.jpg  20.jpg    2851.jpg  3412.jpg  397.jpg   4527.jpg  5075.jpg
1003.txt  1552.txt  20.txt    2851.txt  3412.txt  397.txt   4527.txt  5075.txt
1004.jpg  1553.jpg  2100.jpg  2852.jpg  3413.jpg  3980.jpg  4528.jpg  5076.jpg
1004.txt  1553.txt  2100.txt  2852.txt  3413.txt  3980.txt  4528.txt  5076.txt
1005.jpg  1554.jpg  2101.jpg  2853.jpg  3414.jpg  3981.jpg  4529.jpg  5077.jpg
1005.txt  1554.txt  2101.txt  28

In [ ]:
dir = "Lab5_Data/data/"

from torch.utils.data import Dataset
import numpy as np
import cv2 as cv

In [ ]:
# 写一个类，读取图像
class Image(Dataset):
    def __init__(self, dir, text_examples, is_test=False):
        self.all_image_id = [int(example.id) for example in text_examples]
        self.all_image_paths = [str(dir + example.id + '.jpg') for example in text_examples]
        self.is_test = is_test
        self.all_image_labels = None
        if not self.is_test:
            self.all_image_labels = [example.label for example in text_examples]

    def __getitem__(self, index):
        img = cv.imread(self.all_image_paths[index])
        #img = cv.resize(img, (32, 32))
        img = cv.resize(img, (48, 48))
        img = img / 255
        img = np.transpose(img, [2, 0, 1])
        img = torch.tensor(img, dtype=torch.float32)
        id = self.all_image_id[index]
        id = torch.tensor(id)
        if self.is_test:
            return img, id
        label = self.all_image_labels[index]
        label = torch.tensor(label)
        return img, label, id

    def __len__(self):
        return len(self.all_image_paths)

In [ ]:
# 使用上面写的类处理图像
test_images = Image(dir, test_data.examples, is_test=True)
train_images = Image(dir, train_data.examples)
valid_images = Image(dir, valid_data.examples)
print(test_images[0][0])
print(train_images[0][1])
print(train_images[0][2])
print(valid_images[0][2])

tensor([[[0.3961, 0.3843, 0.3843,  ..., 0.5804, 0.5765, 0.6000],
         [0.3686, 0.3961, 0.4078,  ..., 0.5686, 0.5608, 0.6078],
         [0.3961, 0.3882, 0.4039,  ..., 0.5608, 0.5569, 0.5882],
         ...,
         [0.1333, 0.1373, 0.2588,  ..., 0.2902, 0.2510, 0.2275],
         [0.0980, 0.1843, 0.2549,  ..., 0.4941, 0.7451, 0.3686],
         [0.1176, 0.1686, 0.2078,  ..., 0.6745, 0.4784, 0.2353]],

        [[0.4235, 0.4314, 0.4314,  ..., 0.6392, 0.6196, 0.6118],
         [0.3961, 0.4431, 0.4549,  ..., 0.6275, 0.6000, 0.6235],
         [0.4392, 0.4392, 0.4549,  ..., 0.6039, 0.6000, 0.6118],
         ...,
         [0.0941, 0.0980, 0.2196,  ..., 0.1882, 0.1569, 0.2078],
         [0.0549, 0.1451, 0.2078,  ..., 0.3843, 0.6392, 0.3490],
         [0.0745, 0.1255, 0.1725,  ..., 0.5647, 0.4157, 0.2078]],

        [[0.4392, 0.4314, 0.4314,  ..., 0.6510, 0.6510, 0.6431],
         [0.4078, 0.4431, 0.4549,  ..., 0.6392, 0.6314, 0.6510],
         [0.4392, 0.4314, 0.4471,  ..., 0.6235, 0.6157, 0.

In [ ]:
# 将图像数据输入DataLoader
train_image_iterator = DataLoader(train_images, batch_size=16, shuffle=True)
valid_image_iterator = DataLoader(valid_images, batch_size=16, shuffle=False)
test_image_iterator = DataLoader(test_images, batch_size=16, shuffle=False)

In [ ]:
%ls
# 导入写的两个模型
from model.ImageModel import ImageModel
from model.TextModel import TextModel

data_mining.ipynb  jsonl/  Lab5_Data/  main.ipynb  model/


In [ ]:
print(len(train_iterator))

201


In [ ]:
import torch.nn as nn
import torch.optim as optim

# 训练文本分类器
device = 'cuda'
textmodel = TextModel().to(device=device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(textmodel.parameters(), lr=0.001, momentum=0.5)
num_epochs = 2

for epoch in range(num_epochs):
    running_loss = 0.0
    for batch in train_iterator:
        labels = batch.label.to(device=device, dtype=torch.long)
        inputs = batch.text.t().to(device=device, dtype=torch.float)
        optimizer.zero_grad()
        outputs = textmodel(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print('epoch {0}/{1}, training loss: {2}'.format(epoch+1, num_epochs, running_loss/len(train_iterator)))

# 验证文本分类器效果
correct, total = 0,0

device = 'cuda'
with torch.no_grad():
    for batch in valid_iterator:
        inputs = batch.text.t().to(device=device, dtype=torch.float)
        targets = batch.label.to(device=device, dtype=torch.long)

        outputs = textmodel(inputs)

        _, predicted = torch.max(outputs.data, 1)
        print(predicted)
        total += targets.size(0)
            
        correct += (predicted == targets).sum().item()

print('Accuracy in valid dataset: {0}'.format(correct / total))

epoch 1/2, training loss: 1.224731168343653
epoch 2/2, training loss: 1.090325248478657
tensor([2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2], device='cuda:0')
tensor([2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 1, 1, 2], device='cuda:0')
tensor([2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0')
tensor([0, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 0, 2, 2, 2, 2], device='cuda:0')
tensor([1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0], device='cuda:0')
tensor([2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0')
tensor([2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2], device='cuda:0')
tensor([0, 2, 2, 0, 1, 0, 2, 0, 2, 2, 2, 2, 2, 2, 1, 1], device='cuda:0')
tensor([2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 1, 2, 1], device='cuda:0')
tensor([2, 1, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0')
tensor([1, 2, 2, 1, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0')
tensor([2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 0, 2, 2, 2], device='cuda:0')
tensor([2, 2, 2, 2, 2, 2

In [ ]:
num_epochs = 3

# 训练图像分类器
device = 'cuda'
imagemodel = ImageModel().to(device=device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(imagemodel.parameters(), lr=0.0001, momentum=0.5)

for epoch in range(num_epochs):  
    running_loss = 0.0
    for i, data in enumerate(train_image_iterator, 0):
        inputs, labels, _ = data
        inputs = torch.FloatTensor(inputs).to(device=device)
        labels = torch.LongTensor(labels).to(device=device)

        optimizer.zero_grad()

        outputs = imagemodel(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
    print('epoch {0}/{1}, training loss: {2}'.format(epoch+1, num_epochs, running_loss/len(train_image_iterator)))

# 验证图像分类器效果
correct, total = 0,0

device = 'cuda'
with torch.no_grad():
    for i, data in enumerate(valid_image_iterator, 0):
        inputs, targets, _ = data
        inputs = torch.FloatTensor(inputs).to(device=device)
        targets = torch.LongTensor(targets).to(device=device)

        outputs = imagemodel(inputs)

        _, predicted = torch.max(outputs.data, 1)
        print(predicted)
        total += targets.size(0)
            
        correct += (predicted == targets).sum().item()

print('Accuracy in valid dataset: {0}'.format(correct / total))

epoch 1/3, training loss: 1.1351519360471127
epoch 2/3, training loss: 1.0807504843716598
epoch 3/3, training loss: 1.0663016984118752
tensor([2, 2, 1, 2, 2, 2, 2, 1, 0, 2, 0, 2, 1, 0, 0, 0], device='cuda:0')
tensor([1, 2, 2, 2, 0, 1, 0, 2, 2, 0, 2, 2, 0, 2, 2, 0], device='cuda:0')
tensor([1, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 0], device='cuda:0')
tensor([2, 1, 2, 0, 0, 0, 2, 2, 0, 2, 0, 2, 0, 2, 0, 2], device='cuda:0')
tensor([2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0], device='cuda:0')
tensor([2, 0, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 0, 0, 1, 2], device='cuda:0')
tensor([2, 2, 1, 1, 2, 2, 0, 2, 0, 0, 0, 0, 2, 0, 2, 2], device='cuda:0')
tensor([0, 2, 2, 2, 0, 2, 0, 2, 2, 0, 0, 0, 2, 1, 1, 2], device='cuda:0')
tensor([0, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 0, 2, 2, 2], device='cuda:0')
tensor([0, 0, 0, 0, 2, 2, 2, 2, 2, 1, 2, 0, 0, 2, 0, 2], device='cuda:0')
tensor([2, 2, 2, 1, 0, 2, 0, 2, 2, 2, 0, 1, 2, 0, 1, 2], device='cuda:0')
tensor([0, 2, 2, 2, 0, 2, 0, 2, 0, 0, 2, 0, 2, 1, 2

In [ ]:
# 处理验证集数据，用于融合模型
valid_texts = []
for data in valid_iterator:
    valid_texts.append(data)
valid_images = []
for data in valid_image_iterator:
    valid_images.append(data)
assert len(valid_texts)==len(valid_images)

In [ ]:
for i in range(0, len(valid_texts)):
    print(len(valid_texts[i]))

print('\n')
for i in range(0, len(valid_images)):
    print(len(valid_images[i][0]))

16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
10


16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
10


In [ ]:
#务必判断每一个tensor在data里的id是否对应
for i in range(0, len(valid_texts)):
    text_ids = valid_texts[i].id.to(device='cpu')
    _, _, image_ids = valid_images[i]
    assert text_ids.tolist() == image_ids.tolist()

In [ ]:
# 用验证集输入到融合模型
correct, total = 0,0
device = 'cuda'
with torch.no_grad():
    for i in range(0, len(valid_texts)):
        text_inputs = valid_texts[i].text.t().to(device=device, dtype=torch.float)
        text_targets = valid_texts[i].label.to(device=device, dtype=torch.long)
        text_outputs = textmodel(text_inputs)
        image_inputs, _, _ = valid_images[i]
        image_inputs = torch.FloatTensor(image_inputs).to(device=device)
        image_outputs = imagemodel(image_inputs)

        outputs = 1 * text_outputs + 1 * image_outputs
        _, predicted = torch.max(outputs.data, 1)
        print(predicted)
        total += text_targets.size(0)
        correct += (predicted == text_targets).sum().item()
print('Late Fusion, Accuracy in valid dataset: {0}'.format(correct / total))    

tensor([2, 2, 0, 0, 2, 0, 0, 2, 2, 2, 2, 0, 2, 2, 2, 0], device='cuda:0')
tensor([2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0], device='cuda:0')
tensor([2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2], device='cuda:0')
tensor([2, 1, 0, 0, 2, 2, 0, 2, 2, 2, 2, 2, 0, 0, 2, 2], device='cuda:0')
tensor([2, 2, 2, 1, 0, 2, 2, 0, 2, 1, 2, 2, 2, 1, 2, 2], device='cuda:0')
tensor([2, 2, 0, 1, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 1, 1], device='cuda:0')
tensor([2, 2, 0, 2, 2, 1, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2], device='cuda:0')
tensor([2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2], device='cuda:0')
tensor([2, 2, 2, 2, 2, 0, 0, 1, 2, 2, 2, 2, 2, 0, 2, 2], device='cuda:0')
tensor([2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 1, 2], device='cuda:0')
tensor([2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0')
tensor([1, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2], device='cuda:0')
tensor([2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2], device='cuda:0')
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [ ]:
# 处理测试数据，用于最后用融合模型得出结果
test_texts = []
for data in test_iterator:
    test_texts.append(data)
test_images = []
for data in test_image_iterator:
    test_images.append(data)
assert len(test_texts)==len(test_images)

#务必判断每一个tensor在data里的id是否对应
for i in range(0, len(test_texts)):
    text_ids = test_texts[i].id.to(device='cpu')
    _, image_ids = test_images[i]
    assert text_ids.tolist() == image_ids.tolist()

In [ ]:
# 用测试集得出结果
test_outputs = {}
with torch.no_grad():
    for i in range(0, len(test_texts)):
        text_inputs = test_texts[i].text.t().to(device=device, dtype=torch.float)
        text_outputs = textmodel(text_inputs)
        image_inputs, _ = test_images[i]
        image_inputs = torch.FloatTensor(image_inputs).to(device=device)
        image_outputs = imagemodel(image_inputs)

        outputs = image_outputs + text_outputs
        _, predicted = torch.max(outputs.data, 1)
        print(predicted)
        ids = test_texts[i].id.to(device='cpu').tolist()
        ids = [int(item) for item in ids]
        outputs = predicted.to(device='cpu').tolist()
        for j in range(0, len(ids)):
            test_outputs[str(ids[j])] = index_to_label[outputs[j]]
print(test_outputs)

tensor([2, 1, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0], device='cuda:0')
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 1, 2], device='cuda:0')
tensor([2, 2, 2, 2, 0, 1, 2, 0, 0, 2, 0, 2, 1, 2, 2, 2], device='cuda:0')
tensor([2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 0, 2, 2, 1], device='cuda:0')
tensor([2, 2, 2, 0, 2, 0, 2, 0, 2, 2, 1, 2, 2, 2, 2, 2], device='cuda:0')
tensor([2, 2, 0, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 0, 2, 2], device='cuda:0')
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2], device='cuda:0')
tensor([2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 1, 2, 2], device='cuda:0')
tensor([2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 2], device='cuda:0')
tensor([2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2], device='cuda:0')
tensor([2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0')
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 2, 2, 0], device='cuda:0')
tensor([1, 0, 2, 0, 2, 2, 2, 1, 2, 2, 0, 2, 2, 2, 2, 0], device='cuda:0')
tensor([0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 

In [ ]:
%ls
%ls Lab5_Data/
# 读取要求的输出文本格式
%cat Lab5_Data/test_without_label.txt

data_mining.ipynb  jsonl/  Lab5_Data/  main.ipynb  model/
data/  test_without_label.txt  train.txt
guid,tag
8,null
1576,null
2320,null
4912,null
3821,null
1306,null
4555,null
259,null
3216,null
881,null
1037,null
2463,null
3760,null
3102,null
4074,null
2155,null
5009,null
3292,null
3599,null
1061,null
3927,null
4461,null
716,null
2505,null
1803,null
2157,null
1612,null
4052,null
3641,null
5097,null
2358,null
349,null
4710,null
2991,null
459,null
1737,null
1993,null
116,null
2949,null
2792,null
1580,null
2021,null
4843,null
4827,null
3119,null
3298,null
4541,null
1952,null
1839,null
2127,null
3872,null
3023,null
2351,null
4507,null
600,null
1516,null
4601,null
1046,null
1227,null
4378,null
4357,null
4094,null
5027,null
1969,null
4754,null
861,null
136,null
4466,null
4916,null
1025,null
4922,null
1233,null
1173,null
532,null
51,null
3008,null
1578,null
4945,null
2572,null
173,null
3439,null
755,null
2918,null
1417,null
3152,null
121,null
780,null
4319,null
1191,null
1573,null
1406,null
3

In [ ]:
# 读取要求的文本格式里id的顺序
test_ids = []
with open('Lab5_Data/test_without_label.txt', 'r') as f:
    line  =  f.readline() 
    while line: 
      strs = line.split(',')
      if strs[0] != 'guid':
          test_ids.append(strs[0])
      line = f.readline()
print(test_ids)

['8', '1576', '2320', '4912', '3821', '1306', '4555', '259', '3216', '881', '1037', '2463', '3760', '3102', '4074', '2155', '5009', '3292', '3599', '1061', '3927', '4461', '716', '2505', '1803', '2157', '1612', '4052', '3641', '5097', '2358', '349', '4710', '2991', '459', '1737', '1993', '116', '2949', '2792', '1580', '2021', '4843', '4827', '3119', '3298', '4541', '1952', '1839', '2127', '3872', '3023', '2351', '4507', '600', '1516', '4601', '1046', '1227', '4378', '4357', '4094', '5027', '1969', '4754', '861', '136', '4466', '4916', '1025', '4922', '1233', '1173', '532', '51', '3008', '1578', '4945', '2572', '173', '3439', '755', '2918', '1417', '3152', '121', '780', '4319', '1191', '1573', '1406', '3955', '2904', '2721', '58', '3149', '3690', '3243', '3476', '1393', '3362', '1552', '2348', '1863', '3003', '1949', '2137', '4500', '2823', '1021', '3103', '4981', '3110', '824', '1999', '1769', '4038', '2499', '4287', '4913', '3181', '4578', '1446', '1269', '2803', '1099', '96', '749', 

In [ ]:
# 最后将测试集上的结果输出到文件里，用于提交
with open('test_output.txt', 'w') as f:
    f.write('guid,tag\n')
    for test_id in test_ids:
        f.write('{0},{1}\n'.format(test_id, test_outputs[test_id]))